**RAG with LangChain**

In [1]:
!pip install langchain langchain_openai langchain_experimental pandas numpy tabulate openpyxl -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00


In [2]:
from langchain_openai import ChatOpenAI
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.agents.agent_types import AgentType
from langchain.schema.output_parser import StrOutputParser
from langchain.output_parsers import PandasDataFrameOutputParser
from langchain.prompts import ChatPromptTemplate, PromptTemplate
import pandas as pd
import numpy as np
import os

In [6]:
df= pd.read_excel("titanic.xlsx")


In [7]:
df.head()

,Id_Pasajero,Survived,Clase,Nombre,Sexo,Edad,Familiares,Tiquete,Tarifa,Cabina,Embarcado
0,1,0,3,"Braund, Mr. Owen Harris",Masculino,22.0,0,A/5 21171,7.25,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",Femenino,38.0,0,PC 17599,71.28,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",Femenino,26.0,0,STON/O2. 3101282,7.93,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",Femenino,35.0,0,113803,53.10,C123,S
4,5,0,3,"Allen, Mr. William Henry",Masculino,35.0,0,373450,8.05,NaN,S


In [8]:
os.environ ["OPENAI_API_KEY"] = "###############################################################"
model = "gpt-3.5-turbo-16k"


In [9]:
llm_model = ChatOpenAI(model=model, temperature=0, verbose=True, streaming=True)



In [10]:
df_parser = PandasDataFrameOutputParser(dataframe=df)


In [11]:
print(df_parser.get_format_instructions())

The output should be formatted as a string as the operation, followed by a colon, followed by the column or row to be queried on, followed by optional array parameters.
1. The column names are limited to the possible columns below.
2. Arrays must either be a comma-separated list of numbers formatted as [1,3,5], or it must be in range of numbers formatted as [0..4].
3. Remember that arrays are optional and not necessarily required.
4. If the column is not in the possible columns or the operation is not a valid Pandas DataFrame operation, return why it is invalid as a sentence starting with either "Invalid column" or "Invalid operation".

As an example, for the formats:
1. String "column:num_legs" is a well-formatted instance which gets the column num_legs, where num_legs is a possible column.
2. String "row:1" is a well-formatted instance which gets row 1.
3. String "column:num_legs[1,2]" is a well-formatted instance which gets the column num_legs for rows 1 and 2, where num_legs is a p

La salida debe estar formateada como una cadena con la operación, seguida de dos puntos, seguida de la colurnna o fila a consultar, seguida de parámetros opcionales de arreglo.
1. Los nombres de las columnas se limitan a las posibles columnas a continuación.
2. Los arreglos deben ser una lista de números separados por comas formateados como [1,3,5], o deben estar en un rango de números formateados como [0..4].
3. Recuerda que los arreglos son opcionales y no necesariamente requeridas.
4. Si la columna no está en las columnas posibles o la operación no es una operación válida de Pandas DataFrame, devuelve la razón de por qué es inválida como una oración que comience con "Columna inválida" o "Operación inválida".

Como ejemplo, para los formatos:
1. La cadena "column:num_legs" es un ejemplo bien formateado que obtiene la columna num legs, donde num_legs es una columna posible.
2. La cadena "row:1" es un ejemplo bien formateado que obtiene la fila 1.
3. La cadena "column:num_legs[1,2]" es un ejemplo bien formateado que obtiene la columna num legs para las filas 1 y 2, donde num_legs es una columna posible.
4. La cadena "row:1[num_legs]" es un ejemplo bien formateado que obtiene la fila 1, pero solo para la columna num legs, donde num_legs es una columna
posible.
5. La cadena "mean:num_legs[1..3] es un ejemplo bien formateado que toma la media de num legs desde las filas 1 a 3, donde num legs es una columna
posible y mean es una operación válida de Pandas DataFrame.


In [13]:
prompt = PromptTemplate(
    template = "Responde al usuario su consulta.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": df_parser.get_format_instructions()},
)

In [17]:
prompt = PromptTemplate(
    template = "Responde al usuario su consulta. La respuesta DEBE estar en el formato:\n{format_instructions}\nSolo incluye el comando y los parámetros necesarios. NO incluyas texto adicional.\nConsulta: {query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": df_parser.get_format_instructions()},
)

In [18]:
chain = prompt | llm_model | df_parser

In [19]:
chain.invoke({"query": "dame la primera fila."})

{'0': Id_Pasajero                          1
 Survived                             0
 Clase                                3
 Nombre         Braund, Mr. Owen Harris
 Sexo                         Masculino
 Edad                              22.0
 Familiares                           0
 Tiquete                      A/5 21171
 Tarifa                            7.25
 Cabina                             NaN
 Embarcado                            S
 Name: 0, dtype: object}

In [20]:
chain.invoke({"query": "dame la media de la columna edad."})

{'mean': np.float64(29.69911764705882)}

In [23]:
df["Edad"].mean()

np.float64(29.69911764705882)

In [25]:
pandas_df_agent = create_pandas_dataframe_agent(llm=llm_model, df=df, verbose=True,
                                                handle_parsing_errors=True,
                                                agent_type=AgentType.OPENAI_FUNCTIONS,
                                                return_intermediate_steps=True,
                                                allow_dangerous_code=True)

/usr/local/lib/python3.11/dist-packages/langchain_experimental/agents/agent_toolkits/pandas/base.py:283: UserWarning: Received additional kwargs {'handle_parsing_errors': True} which are no longer supported.
  warnings.warn(


In [26]:
pandas_result = pandas_df_agent({"input": "que tamaño tiene de df?"})

<ipython-input-26-1269879456>:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  pandas_result = pandas_df_agent({"input": "que tamaño tiene de df?"})




> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'df.shape'}`


(891, 11)El dataframe `df` tiene 891 filas y 11 columnas.

> Finished chain.


In [28]:
df.shape

(891, 11)